# 음악 데이터로 트렌드를 읽는 똑똑한 음악 소비자 되기
[배경 이야기] <br>
음악을 사랑하는 대학생 C씨는 멜론에서 최신 인기곡을 자주 듣습니다. 최근 동아리 활동에서 DJ 역할을 맡게 되어, 매주 모임에서 대중이 좋아할 음악을 선곡하는 임무를 받았습니다. 하지만 매번 멜론 앱에서 일일이 인기 차트를 확인하고 선곡을 준비하는 데 시간이 너무 많이 걸려 고민이었습니다.
 
한편, 음악 프로듀서를 꿈꾸는 D씨는 요즘 대중이 어떤 음악을 좋아하는지 데이터로 분석해 새로운 곡을 만들고 싶어합니다. 두 사람은 함께 멜론 차트 데이터를 수집해 대중음악 트렌드를 분석하고 활용하기로 했습니다.
 
### BX : 멜론의 브랜드와 음악 트렌드 신뢰성
- 멜론은 국내 대표 음악 스트리밍 플랫폼으로, 실시간 차트를 통해 대중음악 트렌드를 가장 빠르게 보여줍니다.
### CX : 사용자 맞춤형 음악 데이터 활용
- 매일 실시간으로 바뀌는 인기 차트를 자동으로 수집해, 사용자 경험을 개선합니다.
### DX : 자동화된 음악 데이터 수집과 시각화
-  Requests와 BeautifulSoup를 활용해 멜론의 Top 100 차트를 자동으로 크롤링하고 분석합니다.

In [2]:
import requests as req
from bs4 import BeautifulSoup as bs
import pandas as pd

In [10]:
# 1. 멜론차트 사이트 요청해서 받아오기
res = req.get("https://www.melon.com/chart/index.htm")
res  # [406] ==> 요청에 문제가 있다.(클라이언트 문제_코드로 수정 가능)

<Response [406]>

### 응답코드가 400번대가 나왔다?
- Why? 400번대 응답은 요청이 문제 (서버가 요청하는 방식을 거절당함)
    - 서버가 브라우저가 아닌 코드로 요청 감지
- How? 코드로 요청은 할 건데, 브라우저인척 속이는 작업 -> 브라우저만 가지고 있는 정보를 같이 보내기

In [78]:
# 브라우저를 의미하는 데이터를 제작 -> http요청
header = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36"}
# 브라우저 정보를 담아서 재요청
res = req.get("https://www.melon.com/chart/index.htm", headers=header)
res

<Response [200]>

In [146]:
# 데이터 파싱작업
soup = bs(res.text)
# 순위 데이터 수집
rank = soup.select('td:nth-child(2) > div > span.rank')
# 노래제목 수집
title = soup.select('div.ellipsis.rank01 > span > a')
# 가수이름 수집
singer = soup.select('td:nth-child(6) > div > div > div.ellipsis.rank02 > span')

rank_list = [];title_list = [];singer_list = []
for i in range(100):
    rank_list.append(rank[i].text)
    title_list.append(title[i].text)
    singer_list.append(singer[i].text)

In [148]:
# 데이터 중복여부 검사
for i in singer[:5]:
    print(i.text)

G-DRAGON
로제 (ROSÉ), Bruno Mars
aespa
황가람
로제 (ROSÉ)


In [118]:
# 데이터의 개수가 중요한 경우에는 반드시 개수를 검증하자
print(len(rank),len(title),len(singer))  # rank 0 : 순위

100 100 100


In [122]:
print(rank[0].text,title[0].text,singer[0].text)

1 HOME SWEET HOME (feat. 태양, 대성) G-DRAGON


In [150]:
data = {'순위':rank_list,'노래제목':title_list,'가수이름':singer_list}

In [152]:
song_df = pd.DataFrame(data)
song_df

,순위,노래제목,가수이름
0,1,"HOME SWEET HOME (feat. 태양, 대성)",G-DRAGON
1,2,APT.,"로제 (ROSÉ), Bruno Mars"
2,3,Whiplash,aespa
3,4,나는 반딧불,황가람
4,5,toxic till the end,로제 (ROSÉ)
...,...,...,...
95,96,Nice Guy,BOYNEXTDOOR
96,97,HERO (From the Film “소방관”),박효신
97,98,가까운 듯 먼 그대여,카더가든
98,99,눈사람,정승환


In [128]:
song_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   순위      100 non-null    object
 1   노래제목    100 non-null    object
 2   가수이름    100 non-null    object
dtypes: object(3)
memory usage: 2.5+ KB


In [154]:
song_df.to_csv('멜론차트Top100수집.csv', index=False,encoding='utf-8-sig')

## 이번파일 정리
- req로 요청 보냈을 때 400번대 응답이 나오면 브라우저 정보 담아서 보내주기
- 데이터를 수집할 때 개수가 중요하면 반드시 개수, 중복여부 검증해보기
- 개발자도구의 element탭에 있는 테그들을 분석 -> 주변에 있는 태그들을 분석해보자